In [ ]:
import random
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
from utils.data_loader import create_dataset
from utils.openmax import get_activations, compute_openmax
from utils.openmax_utils import get_openmax_predict, get_openmax_label, convert_to_binary_label, convert_to_label, compute_roc, plot_roc, compute_pr, plot_pr
from utils.graph import draw_confusion_matrix

In [ ]:
print('Num GPUs Available: ', len(tf.config.list_physical_devices('GPU')))

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        print(e)

In [ ]:
SEED = 0
IMG_DIM = 299
BATCH_SIZE = 16
FOLD = 5

In [ ]:
DATA_CSV = f'data/datasplit_fold{FOLD}.csv'

In [ ]:
random.seed(SEED)
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [ ]:
df = pd.read_csv(DATA_CSV)

In [ ]:
model = tf.keras.models.load_model(f'models/fold_{FOLD}.h5')

In [ ]:
test_unknown_ds = create_dataset(df, BATCH_SIZE, 'test_unknown')
_, y_test_unknown = test_unknown_ds.get_all()

In [ ]:
logits_test, softmax_test = get_activations(test_unknown_ds, model)

In [ ]:
openmax = []
prob_u = []
for logits in logits_test:
    temp_openmax, temp_prob_u = compute_openmax(logits.reshape(1, -1), FOLD)
    openmax.append(temp_openmax)
    prob_u.append(temp_prob_u)

openmax = np.array(openmax)
prob_u = np.array(prob_u)

In [ ]:
y_test_unknown_binary = convert_to_binary_label(y_test_unknown)

In [ ]:
roc = compute_roc(y_test_unknown_binary, prob_u)

In [ ]:
pr = compute_pr(y_test_unknown_binary, prob_u)

In [ ]:
plot_roc(roc)

In [ ]:
plot_pr(pr)

In [ ]:
roc_thresholds = roc['thresholds']
np.savetxt('roc_thresholds.csv', roc_thresholds, delimiter=',')

In [ ]:
pr_thresholds = pr['thresholds']
np.savetxt('pr_thresholds.csv', pr_thresholds, delimiter=',')

In [ ]:
y_test_unknown_label = convert_to_label(y_test_unknown)

In [ ]:
precision_arr = []
recall_arr = []
f1_arr = []
accuracy_arr = []
for threshold in pr_thresholds:
    y_hat_test = []
    for probs in openmax:
        temp = get_openmax_label(probs, threshold=threshold)
        y_hat_test.append(temp)
    y_hat_test = np.asarray(y_hat_test)

    precision = precision_score(y_test_unknown_label, y_hat_test, average='macro')
    recall = recall_score(y_test_unknown_label, y_hat_test, average='macro')
    f1 = f1_score(y_test_unknown_label, y_hat_test, average='macro')
    accuracy = accuracy_score(y_test_unknown_label, y_hat_test)

    precision_arr.append(precision)
    recall_arr.append(recall)
    f1_arr.append(f1)
    accuracy_arr.append(accuracy)

In [ ]:
best_idx = np.argmax(f1_arr)

In [ ]:
precision_arr[best_idx]

In [ ]:
recall_arr[best_idx]

In [ ]:
f1_arr[best_idx]

In [ ]:
accuracy_arr[best_idx]

In [ ]:
# test_ds = create_dataset(df, BATCH_SIZE, 'test')
# x_test, y_test = test_ds.get_all()

In [ ]:
# for i in range(5):
#     random_char = np.random.randint(0, len(x_test))

#     test_x1 = x_test[random_char]
#     test_y1 = y_test[random_char]

#     logits, softmax = get_activations(
#         test_x1.reshape(-1, IMG_DIM, IMG_DIM, 3), model)

#     openmax, _ = compute_openmax(logits, FOLD)

#     print(f'SoftMax Sum: {np.sum(softmax)}')
#     print(f'OpenMax Sum: {np.sum(openmax)}')
#     print(f'True Label: {np.argmax(test_y1)}')
#     print(f'SoftMax Label: {np.argmax(softmax)}')
#     print(f'OpenMax Label: {get_openmax_label(openmax, pr_thresholds[best_idx])}')
#     plt.imshow((test_x1+1)/2)
#     plt.show()

In [ ]:
# y_hat = model.predict(test_ds)

In [ ]:
# y_hat = np.argmax(y_hat, axis=1).astype(np.uint8)
# y_test = np.argmax(y_test, axis=1).astype(np.uint8)

In [ ]:
# cm = confusion_matrix(y_test, y_hat, normalize='true')
# cm = np.round(cm, 2)

In [ ]:
# class_name = range(test_ds.get_class_num())

In [ ]:
# draw_confusion_matrix(cm, class_name)